In [1]:
# imports
import numpy as np
from PIL import Image

In [2]:
# variables
width = 600
height = 400
viewportSize = 1
projectionPlane = 1

#Scene objects

camera = np.array([0,0,0])

spheres = [{"center": np.array([0,-1,3]), "radius": 1, "color": (255,0,0),"S":500, "refl": 0.0, "refr": 0},#red
          {"center": np.array([-2,0,4]), "radius": 1, "color": (0,255,0),"S":10 , "refl": 0,  "refr": 0}, #green
          {"center": np.array([2,0,4]), "radius": 1, "color": (0,0,255),"S":500,  "refl": 0.1,  "refr": 0}, #blue
          {"center": np.array([0,-5001,0]), "radius": 5000, "color": (255,255,0),"S":0,  "refl": 0 , "refr": 0} #grey floor
          ]
lights = [
    {"type": "ambient","intensity": 0.2},
    {"type": "point","intensity": 0.6, "position": np.array([2,1,0])},
    {"type": "directional","intensity": 0.2, "direction": np.array([1,4,4])}
]

In [3]:
def canvasViewport(x,y):
    return np.array([
        x * viewportSize / width,
        y * viewportSize / height,
        projectionPlane
    ])

In [4]:
def intersectRays(origin, direction, sphere):
    
    center = sphere["center"]
    r = sphere["radius"]
    oc = origin - center
    
    a = np.dot(direction,direction)
    b = 2 * np.dot(oc,direction)
    c = np.dot(oc,oc) - r**2
    
    disc = b**2 - 4*(a*c)
    if (disc < 0):
        return float("inf"), float("inf")
    else:
        t1 = (-b - np.sqrt(disc)) / (2*a)
        t2 = (-b + np.sqrt(disc)) / (2*a)
        return t1,t2
    
    

In [5]:
def closestIntersection(O,D,t_min):
    closest_t = float("inf") 
    closestSphere = None
        
    #check for ray intersections with spheres
    for sphere in spheres:
        t1,t2 = intersectRays(O, D, sphere)
            
        if t1> t_min and t1 < closest_t:
            closest_t = t1
            closestSphere = sphere
            pixelColor = closestSphere["color"]
                
        if t2 > t_min and t2 < closest_t:
            closest_t = t2
            closestSphere = sphere
            pixelColor = closestSphere["color"]
            
    return closestSphere, closest_t
        

In [6]:
def computeLighting (P,N,V,S):
    intensity = 0.0
    
    for light in lights:
        if light["type"]=="ambient":
            intensity = intensity + light["intensity"]
        else:
            if light["type"] == "point":
                L = light["position"] - P
            else:
                L = light["direction"]
                
            L = L/np.linalg.norm(L)
            
            shadowSphere, shadow_t = closestIntersection(P,L,0.001)
            if shadowSphere != None:
                return min(intensity,1)
            
            n_dot_l = np.dot(N,L)
            if n_dot_l > 0:
                intensity = intensity + light["intensity"] * n_dot_l / (np.linalg.norm(L) * np.linalg.norm(N))
                
            if S != -1:
                R = 2 * N * np.dot(N,L) - L
                r_dot_v = np.dot(R,V)
                if r_dot_v > 0:
                    intensity = intensity + light["intensity"] * (r_dot_v/(np.linalg.norm(R)*np.linalg.norm(V))) ** S
                    
            
            
    return min(intensity,1)

In [7]:
def reflectRay(R,N):
    return 2 * N * np.dot(N,R) - R

In [8]:
def traceRay(O, D, t_min, t_max, depth):
    closestS, closestT = closestIntersection(O, D, t_min)

    if closestS == None: 
        return  (158, 224, 255)
    P = O + closestT * D
    N = P - closestS["center"]
    N = N/np.linalg.norm(N)
    pixelColor = closestS["color"]
    localColor = tuple(map(lambda v: min(round(v * computeLighting(P,N,-D,closestS["S"])),255), pixelColor))
    
    refr = closestS["refr"]
    

    refl = closestS["refl"]
    if depth<= 0 or refl <= 0:
        return localColor

    R = reflectRay(-D, N)
        
    reflectedColor = traceRay(P, R, 0.001, t_max, depth - 1) 
    
    

    temp1 = tuple(map(lambda v: v * (1 - refl), localColor))
    temp2 = tuple(map(lambda v: v * refl, reflectedColor))
    return tuple(map(lambda v1,v2: min(round(v1 + v2 ),255), temp1, temp2))

In [ ]:
image = Image.new("RGB", (width, height), "white")
pixels = image.load()

#create canvas
for x in range(width):
    #print( x/width * 100, "%")
    for y in range(height):
        viewX = (x - width/2)
        viewY = -(y - height/2)
        direction = canvasViewport(viewX,viewY)
        direction = direction /np.linalg.norm(direction)
        closest_t = float("inf")
        #pixelColor = (255,255,255) #background color if no intersection
        intersection = False
        
        #check for ray intersections with spheres

        pixelColor = traceRay(camera, direction, 1, 1000, 3)
        pixels[x,y] = pixelColor
        

In [ ]:
image.show()